# 🧠 Grafana Dashboard Summarizer
Simulate reading a Grafana dashboard JSON and summarize its panels using GPT or plain logic.

In [0]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI
import json
import pandas as pd

In [0]:


with open("mock_grafana_dashboard.json", "r") as f:
    data = json.load(f)

dashboard = data["dashboard"]
panels = dashboard["panels"]
print(f"Dashboard Title: {dashboard['title']}")
print(f"Total Panels: {len(panels)}\n")
for p in panels:
    print(f"- {p['title']} ({p['type']})")


In [0]:
# Extracting panel data

panel_data = []
for p in panels:
    thresholds = p.get("fieldConfig", {}).get("defaults", {}).get("thresholds", {}).get("steps", [])
    panel_data.append({
        "Title": p["title"],
        "Type": p["type"],
        "Unit": p.get("fieldConfig", {}).get("defaults", {}).get("unit", "N/A"),
        "Thresholds": thresholds
    })

df = pd.DataFrame(panel_data)
df


In [0]:

summary_prompt = f"""
You are a helpful assistant summarizing a system monitoring dashboard.

Dashboard: {dashboard['title']}
Panels:
"""
for idx, row in df.iterrows():
    summary_prompt += f"- {row['Title']} [{row['Type']}] - Unit: {row['Unit']}, Thresholds: {row['Thresholds']}\n"

print(summary_prompt)


In [0]:


load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


In [0]:
openai = OpenAI()

In [0]:
def summarize():
    response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a Grafana dashboard summarizer."},
        {"role": "user", "content": summary_prompt}
    ]
)
    return response.choices[0].message.content

In [0]:

summary = summarize()
display(Markdown(summary))